In [31]:
import pandas as pd

# Пример данных за предыдущий год и текущий год
df_previous_year = pd.read_excel('свод_12 мес 2023.xlsx')
df_current_year = pd.read_excel('свод_9 мес 2024.xlsx')

df_previous_year[['Месяц', 'Наименование дирекции по энергообеспечению', 'Субъект РФ', 'Поставщик', 'Вид потребления (электроэнергия / мощность)', 'Группа', 'Уровень напряжения', 'ЦК', 'Цена', 'Т усл. (2ст.)']] = df_previous_year[['Месяц', 'Наименование дирекции по энергообеспечению', 'Субъект РФ', 'Поставщик', 'Вид потребления (электроэнергия / мощность)', 'Группа', 'Уровень напряжения', 'ЦК', 'Цена', 'Т усл. (2ст.)']].fillna('Неизвестно')
df_current_year[['Месяц', 'Наименование дирекции по энергообеспечению', 'Субъект РФ', 'Поставщик', 'Вид потребления (электроэнергия / мощность)', 'Группа', 'Уровень напряжения', 'ЦК', 'Цена', 'Т усл. (2ст.)']] = df_current_year[['Месяц', 'Наименование дирекции по энергообеспечению', 'Субъект РФ', 'Поставщик', 'Вид потребления (электроэнергия / мощность)', 'Группа', 'Уровень напряжения', 'ЦК', 'Цена', 'Т усл. (2ст.)']].fillna('Неизвестно')

# Словарь для преобразования названий месяцев на русском в названия месяцев на английском
month_mapping = {
    'январь': 'January',
    'февраль': 'February',
    'март': 'March',
    'апрель': 'April',
    'май': 'May',
    'июнь': 'June',
    'июль': 'July',
    'август': 'August',
    'сентябрь': 'September',
    'октябрь': 'October',
    'ноябрь': 'November',
    'декабрь': 'December'
}

# Ф-ция для разделения строки на месяц и год для преобразования в формат даты
def parse_date(date_str):
    month, year = date_str.split()
    month_name = month_mapping[month]
    return pd.to_datetime(f"{year}-{month_name}")

# Преобразование названий месяцев
df_previous_year['Дата'] = df_previous_year['Месяц и Год'].apply(parse_date)
df_current_year['Дата'] = df_current_year['Месяц и Год'].apply(parse_date)

# Упорядочивание значений в столбце Месяц и Год внутри каждой дирекции и субъекта
df_current_year.sort_values(['Наименование дирекции по энергообеспечению', 'Субъект РФ', 'Поставщик', 'Вид потребления (электроэнергия / мощность)', 'Группа', 'Уровень напряжения', 'ЦК', 'Цена', 'Т усл. (2ст.)', 'Дата'], inplace=True)
df_current_year.drop(['Месяц и Год'], axis=1, inplace=True)

# Функция для выявления аномалий
def detect_anomalies(group, previous_mean):
    anomalies = []
    for i in range(1, len(group)):
        current_value = group.iloc[i]['Объемы']
        previous_value = group.iloc[i-1]['Объемы']
        # Проверка, что текущее значение не отличается от предыдущего больше чем в 10 раз
        if (abs(current_value) > 10 * abs(previous_value) or abs(current_value) < 0.1 * abs(previous_value)):# or \
           #(abs(current_value) > 10 * abs(previous_mean) or abs(current_value) < 0.1 * abs(previous_mean)):
            anomalies.append(group.iloc[i])
    return pd.DataFrame(anomalies)

# Группировка данных
grouped_previous_year = df_previous_year.groupby(['Наименование дирекции по энергообеспечению', 'Субъект РФ', 'Поставщик', 'Вид потребления (электроэнергия / мощность)', 'Группа', 'Уровень напряжения', 'Цена', 'ЦК', 'Т усл. (2ст.)'])
grouped_current_year = df_current_year.groupby(['Наименование дирекции по энергообеспечению', 'Субъект РФ', 'Поставщик', 'Вид потребления (электроэнергия / мощность)', 'Группа', 'Уровень напряжения', 'Цена', 'ЦК', 'Т усл. (2ст.)'])

# Применение функции для выявления аномалий
anomalies = pd.DataFrame()

for name, group in grouped_current_year:
    if name in grouped_previous_year.groups:
        previous_group = grouped_previous_year.get_group(name)
        previous_mean = previous_group['Объемы'].mean()
    else:
        previous_mean = None

    # Перебор по месяцам внутри каждой группы
    if previous_mean is not None:
        group_anomalies = detect_anomalies(group, previous_mean)
        if not group_anomalies.empty:
            anomalies = pd.concat([anomalies, group_anomalies])

# Вывод аномалий
print("Аномалии:")
print(anomalies)

# Сохранение аномалий в Excel
anomalies.to_excel('anomalies.xlsx', index=False)

Аномалии:
          Месяц        Наименование дирекции по энергообеспечению  \
26292   февраль  Восточно-Сибирская дирекция по энергообеспечению   
26679       май  Восточно-Сибирская дирекция по энергообеспечению   
26681       май  Восточно-Сибирская дирекция по энергообеспечению   
26600    апрель  Восточно-Сибирская дирекция по энергообеспечению   
26604    апрель  Восточно-Сибирская дирекция по энергообеспечению   
...         ...                                               ...   
14461    август   Северо-Кавказская дирекция по энергообеспечению   
15281  сентябрь   Северо-Кавказская дирекция по энергообеспечению   
15341    апрель       Юго-Восточная дирекция по энергообеспечению   
21983      март      Южно-Уральская дирекция по энергообеспечению   
22505      июнь      Южно-Уральская дирекция по энергообеспечению   

                 Субъект РФ                                       Поставщик  \
26292     Иркутская область         ООО "Иркутская Энергосбытовая компания"   
266